In [1]:
import numpy as np 
import cmocean
from scipy import ndimage
from netCDF4 import Dataset
import glob
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
import datetime 
from datetime import date 
import mpl_toolkits.basemap as bm
import matplotlib.animation as animation
import numpy.ma as ma
import matplotlib.cm as cm
from matplotlib.patches import Polygon
from scipy import signal, stats
import time
from mpl_toolkits.axes_grid1 import make_axes_locatable
import warnings; warnings.simplefilter('ignore')

from functions import load_ice, trend, find_trends, plot_cartopy, decomp


path = '/extra-space1/data/tikoralukupload/cis-weekly/nc/'
region = 'EC'

# Projection for mapping
rot = ccrs.RotatedPole(pole_latitude=55,pole_longitude=150)

# Read in data
nc = Dataset(path + region + '.nc', 'r')
lon = nc.variables['longitude'][:1200,:1500]
lat = nc.variables['latitude'][:1200,:1500]
juld = nc.variables['juld'][:]
land = nc.variables['land'][:1200,:1500]
E_CT = nc.variables['E_CT'][:][:,:1200,:1500]

# Date time information
d0ord = date(1950,1,1).toordinal()
dt_ordinal = d0ord + juld
dates = [date.fromordinal(dt_ordinal[tt]) for tt in range(len(juld))]
months = [dates[tt].month for tt in range(len(juld))]
years = [dates[tt].year for tt in range(len(juld))]
months_unique = np.unique(months)
years_unique = np.unique(years)

In [2]:
# Set ocean values to 0
E_CT_masked = E_CT.copy()
E_CT_masked[E_CT_masked.mask == True] = 0
# Make land array same shape as data
land3d = np.repeat(land[np.newaxis, :, :], len(E_CT), axis=0)
# mask out land using land array
masked_array = np.ma.masked_array(E_CT_masked, land3d)
masked_array.filled(fill_value=255)

In [ ]:
plt.contourf(masked_array)

In [ ]:
np.unique(masked_array)

In [51]:
days = [1586, 1587, 1588, 1589]

day = 1586

# for day in days:

field = masked_array[day].data

# Range of concentration limits
ssh_crits = np.arange(0,5,1)
i_cen_arr = []
j_cen_arr = []
area_polynyas = []

for ssh_crit in ssh_crits:
	regions, nregions = ndimage.label( (field<ssh_crit).astype(int) )
	for iregion in range(nregions):

		# 1. set size min/max
		# must be at least Npix_min pixels
		Npix_min = 30
		Npix_max = 99999999999
		region = (regions==iregion+1).astype(int)
		region_Npix = region.sum()
		# Is polynya area within size limits T/F
		polynya_area_within_limits = ((region_Npix >= Npix_min) * (region_Npix <= Npix_max))			

		# 2. crit 2
		crit2result = True
		# 3. crit 3
		crit3result = True

		# Quit loop if these are not satisfied
		if np.logical_not(polynya_area_within_limits * crit2result * crit3result ):
			p = 0
			continue

		if (polynya_area_within_limits * crit2result * crit3result ):

			# save the location of the centre of the polynya
			polynya_object_with_mass = field * region
			polynya_object_with_mass[np.isnan(polynya_object_with_mass)] = 0
			j_cen, i_cen = ndimage.center_of_mass(polynya_object_with_mass)

			# create arrays of lon/lat indices. Index them as lon[j,i]
			i_cen_arr = np.append(i_cen_arr, np.round(int(i_cen)))
			j_cen_arr = np.append(j_cen_arr, np.round(int(j_cen)))

			area = region_Npix  # [km**2]
			area_polynyas = np.append(area_polynyas, area)   


plt.figure()
plt.contourf(lon, lat, field)
plt.plot(lon[j_cen_arr], lat[i_cen_arr])

hello
[[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]]


ValueError: cannot convert float NaN to integer